# STATS 101C Final Project Notebook

Goal is to construct a new dataset with better, more "informational features" that we can then plug into a model for predicting whether a team would win over another

In [1]:
# reading data
import pandas as pd

In [7]:
# need openpyxl package installed
df = pd.read_excel("Dataset.xlsx")
df.head()

,Team,Match Up,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,GSW,GSW vs. PHX,10/24/2023,L,240,104,36,101,35.6,10,...,78.6,18,31,49,19,11,6,11,23,-4
1,PHX,PHX @ GSW,10/24/2023,W,240,108,42,95,44.2,11,...,76.5,17,43,60,23,5,7,19,22,4
2,LAL,LAL @ DEN,10/24/2023,L,240,107,41,90,45.6,10,...,75,13,31,44,23,5,4,12,18,-12
3,DEN,DEN vs. LAL,10/24/2023,W,240,119,48,91,52.7,14,...,75,9,33,42,29,9,6,12,15,12
4,MEM,MEM vs. NOP,10/25/2023,L,240,104,38,91,41.8,12,...,80,8,29,37,23,8,7,13,19,-7


### Creating Better Features - Column for Opposing Team, Column for Home Game (Yes/No)

In [ ]:
# need to provide a feature for indicating whether it was a home game for the team, or not
home_game = df["Match Up"].apply(lambda x: 0 if "@" in x else 1)

# print to see if it looks accurate (1 for home game, 0 for away game)
pd.concat([df["Match Up"], home_game], axis=1)

,Match Up,Match Up
0,GSW vs. PHX,1
1,PHX @ GSW,0
2,LAL @ DEN,0
3,DEN vs. LAL,1
4,MEM vs. NOP,1
...,...,...
2455,CLE vs. CHA,1
2456,NYK vs. CHI,1
2457,LAC vs. HOU,1
2458,LAL @ NOP,0


In [9]:
# need column for just the name of the opposing team (last 3 characters)
opposing_team = df.iloc[:, 1].str[-3:]

# check for equality to make sure didn't mess it up
all(opposing_team == (df["Match Up"].str[-3:]))

True

In [10]:
# remove "Match Up" column
df.drop("Match Up", axis = 1, inplace = True)

In [11]:
# replace w/ new columns
df.insert(1, "opposing_team", opposing_team)
df.insert(2, "home_game", home_game)
df.head()

,Team,opposing_team,home_game,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,GSW,PHX,1,10/24/2023,L,240,104,36,101,35.6,...,78.6,18,31,49,19,11,6,11,23,-4
1,PHX,GSW,0,10/24/2023,W,240,108,42,95,44.2,...,76.5,17,43,60,23,5,7,19,22,4
2,LAL,DEN,0,10/24/2023,L,240,107,41,90,45.6,...,75,13,31,44,23,5,4,12,18,-12
3,DEN,LAL,1,10/24/2023,W,240,119,48,91,52.7,...,75,9,33,42,29,9,6,12,15,12
4,MEM,NOP,1,10/25/2023,L,240,104,38,91,41.8,...,80,8,29,37,23,8,7,13,19,-7


### Selecting Data Before Nov 13, 2023

In [12]:
# view the earliest date and latest date in the entire dataset
earliest_date = pd.to_datetime(df["Game Date"]).min()
print("Earliest Date:" , earliest_date.strftime('%B %d, %Y'))
latest_date = pd.to_datetime(df["Game Date"]).max()
print("Latest Date:", latest_date.strftime('%B %d, %Y'))

Earliest Date: October 24, 2023
Latest Date: April 14, 2024


In [ ]:
# create a new dataset with data before Nov 13, 2023
cutoff_date = "2023-11-13"
before = df[pd.to_datetime(df["Game Date"]) < pd.to_datetime(cutoff_date)]

# view the earliest date and latest date from the new dataset
earliest_date = pd.to_datetime(before["Game Date"]).min()
latest_date = pd.to_datetime(before["Game Date"]).max()
print("Earliest Date:" , earliest_date.strftime('%B %d, %Y'))
print("Latest Date:", latest_date.strftime('%B %d, %Y'))

Earliest Date: October 24, 2023
Latest Date: November 12, 2023


### Creating Better Features - Stability of Key Game Statistics

In [14]:
# let's work with only a single team for now
gsw = before[before["Team"] == "GSW"]
gsw

,Team,opposing_team,home_game,Game Date,W/L,MIN,PTS,FGM,FGA,FG%,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,GSW,PHX,1,10/24/2023,L,240,104,36,101,35.6,...,78.6,18,31,49,19,11,6,11,23,-4
35,GSW,SAC,0,10/27/2023,W,240,122,48,87,55.2,...,85.7,7,32,39,32,14,3,19,24,8
69,GSW,HOU,0,10/29/2023,W,240,106,35,81,43.2,...,78.3,8,41,49,27,6,6,14,18,11
94,GSW,NOP,0,10/30/2023,W,240,130,51,98,52.0,...,65,21,43,64,28,6,7,15,21,28
116,GSW,SAC,1,11/01/2023,W,240,102,39,81,48.1,...,86.7,7,29,36,32,5,6,18,17,1
154,GSW,OKC,0,11/03/2023,W,240,141,49,92,53.3,...,83.3,13,25,38,36,9,2,15,25,2
180,GSW,CLE,0,11/05/2023,L,240,104,34,94,36.2,...,66.7,17,27,44,26,10,4,13,18,-11
195,GSW,DET,0,11/06/2023,W,240,120,46,97,47.4,...,76.2,17,27,44,26,8,4,11,14,11
214,GSW,DEN,0,11/08/2023,L,240,105,37,86,43.0,...,76.9,11,38,49,21,5,0,9,20,-3
262,GSW,CLE,1,11/11/2023,L,240,110,37,90,41.1,...,76.7,15,40,55,26,5,3,20,29,-8


In [15]:
# get a dataframe of only the game stats
gsw_stats = gsw.iloc[:, 5:]

# calculate the stability of the game stats

# get the average
avg_gsw_stats = gsw_stats.mean()

# get the std dev
sd_gsw_stats = gsw_stats.std()

# get the coefficient of variation
coeff_of_var = sd_gsw_stats / avg_gsw_stats

# take the average of the coefficients of variation
avg_variation = coeff_of_var.mean()

# subtract from 1 to get a measure of "stability", closer to 1 = more stable, closer to 0 = less stable
stability_score = 1 - avg_variation
stability_score

np.float64(0.5879039783343367)

In [ ]:
# creating a running stability score, that gives us the stability score of the team up until that date
# basically code above, just done on a rolling basis
# needs at least two rows of data from above, since current row data doesn't count, and 2 data points needed for std calculation
gsw_stats["stability_score"] = gsw_stats.shift(1).expanding().apply(lambda x: 1 - (x.std() / x.mean())).mean(axis = 1)
gsw_stats

,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,stability_score
0,240,104,36,101,35.6,10,43,23.3,22,28,...,18,31,49,19,11,6,11,23,-4,NaN
35,240,122,48,87,55.2,14,32,43.8,12,14,...,7,32,39,32,14,3,19,24,8,NaN
69,240,106,35,81,43.2,18,45,40.0,18,23,...,8,41,49,27,6,6,14,18,11,0.549712
94,240,130,51,98,52.0,15,41,36.6,13,20,...,21,43,64,28,6,7,15,21,28,0.705880
116,240,102,39,81,48.1,11,31,35.5,13,15,...,7,29,36,32,5,6,18,17,1,0.738931
154,240,141,49,92,53.3,18,40,45.0,25,30,...,13,25,38,36,9,2,15,25,2,0.731380
180,240,104,34,94,36.2,16,41,39.0,20,30,...,17,27,44,26,10,4,13,18,-11,0.725856
195,240,120,46,97,47.4,12,41,29.3,16,21,...,17,27,44,26,8,4,11,14,11,0.686382
214,240,105,37,86,43.0,11,35,31.4,20,26,...,11,38,49,21,5,0,9,20,-3,0.714599
262,240,110,37,90,41.1,13,38,34.2,23,30,...,15,40,55,26,5,3,20,29,-8,0.692215


### Creating Better Features: Weighted Average

In [155]:
import numpy as np

In [186]:
# drop stability score, since weighted average is not computed using this
gsw_stats.drop("stability_score", axis = 1, inplace = True)

In [ ]:
# create a weighted average function, that weights more recent games higher
def weighted_average(window):
    weights = np.linspace(1, 2, num = len(window)) # compute weights
    weights = weights / weights.sum() # normalize between 0 and 1
    return np.average(window, weights = weights) # take weighted average

# get rolling weighted average of game stats for Golden State Warriors
# need to manual index, since np.average fails on NaN values, and shifting by 1 always gives a NaN column at the top
gsw_weighted_average = gsw_stats.shift(1).expanding().apply(lambda window: weighted_average(window[1:])) 
gsw_weighted_average

,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,240.0,104.000000,36.000000,101.000000,35.600000,10.000000,43.000000,23.300000,22.000000,28.000000,78.600000,18.000000,31.000000,49.000000,19.000000,11.000000,6.000000,11.000000,23.000000,-4.000000
69,240.0,116.000000,44.000000,91.666667,48.666667,12.666667,35.666667,36.966667,15.333333,18.666667,83.333333,10.666667,31.666667,42.333333,27.666667,13.000000,4.000000,16.333333,23.666667,4.000000
94,240.0,110.888889,39.555556,87.444444,45.511111,14.888889,40.222222,37.555556,16.888889,21.111111,80.833333,9.888889,35.777778,45.666667,26.888889,9.777778,5.000000,15.000000,21.111111,6.666667
116,240.0,117.222222,43.388889,91.333333,47.533333,14.777778,40.444444,36.927778,15.666667,20.833333,75.561111,13.777778,38.000000,51.777778,27.111111,8.611111,5.666667,14.944444,21.166667,13.500000
154,240.0,112.933333,42.100000,88.633333,47.546667,13.700000,37.900000,36.413333,15.033333,19.333333,78.710000,11.933333,35.433333,47.366667,28.333333,7.733333,5.733333,15.733333,20.100000,9.800000
180,240.0,119.155556,43.600000,89.488889,48.744444,14.644444,38.422222,38.257778,17.311111,21.777778,79.746667,12.200000,33.088889,45.288889,29.955556,8.088889,4.888889,15.533333,21.244444,7.955556
195,240.0,116.111111,41.714286,90.396825,46.274603,14.873016,38.936508,38.314286,17.809524,23.333333,77.247619,13.126984,31.968254,45.095238,29.111111,8.492063,4.730159,15.031746,20.634921,4.317460
214,240.0,116.714286,42.428571,91.511905,46.458333,14.369048,39.285714,36.752381,17.488095,22.904762,77.115476,13.761905,31.190476,44.952381,28.547619,8.428571,4.619048,14.357143,19.547619,5.488095
262,240.0,114.935185,41.611111,90.685185,45.939815,13.861111,38.648148,35.945370,17.851852,23.342593,77.116667,13.333333,32.250000,45.583333,27.407407,7.935185,3.944444,13.574074,19.648148,4.250000


## Apply Feature Creation for Final Dataset

In [283]:
# get name of columns with only game statistics for easy reference (implicitly removed the MIN column)
game_stats_columns = before.columns[6:]
game_stats_columns

Index(['PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-'],
      dtype='object')

### Create Stability Score Feature

In [284]:
# create the "stability score feature"
stability_score_feature = (
    before.groupby("Team") # group by team
    .apply(lambda group: group.sort_values(by = "Game Date")[game_stats_columns] # within each group, sort by game date and use only stats cols
           .shift(1).expanding().apply(lambda window: 1 - (window.std() / window.mean())).mean(axis = 1) # rolling calc of stability
        )
    .reset_index(level = 0, drop = True) # makes the original index (level 0) be the index of the column, not the group-level index
)
stability_score_feature.name = "stability_score"

C:\Users\steph\AppData\Local\Temp\ipykernel_13396\1619578001.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.sort_values(by = "Game Date")[game_stats_columns] # within each group, sort by game date and use only stats cols


### Create the Difference in Weighted Average Feature

In [285]:
# compute rolling weighted average game stats for each team

rolling_weighted_average_game_stats = (
    before.groupby("Team") # group by team
    .apply(lambda group: group.sort_values(by = "Game Date")[game_stats_columns]
           .shift(1).expanding().apply(lambda window: weighted_average(window[1:]))
        )
    .reset_index(level = 0, drop = True)
)
rolling_weighted_average_data = pd.concat([before[["Team", "opposing_team", "Game Date"]], rolling_weighted_average_game_stats], axis = 1)
rolling_weighted_average_data.tail()

C:\Users\steph\AppData\Local\Temp\ipykernel_13396\2096523324.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group.sort_values(by = "Game Date")[game_stats_columns]


,Team,opposing_team,Game Date,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
281,NYK,CHA,11/12/2023,107.797619,38.785714,92.583333,41.930952,12.809524,36.916667,33.930952,...,74.710714,15.035714,35.011905,50.047619,22.964286,8.773810,1.702381,13.309524,17.261905,5.583333
282,WAS,BKN,11/12/2023,119.773810,45.190476,92.202381,49.103571,13.464286,36.511905,37.273810,...,73.942857,8.988095,29.047619,38.035714,28.654762,8.297619,5.892857,14.761905,20.642857,-6.535714
283,PHX,OKC,11/12/2023,113.972222,40.092593,87.287037,46.101852,13.083333,35.064815,37.176852,...,81.182407,11.851852,34.277778,46.129630,25.861111,7.916667,6.851852,16.555556,17.981481,0.657407
284,LAC,MEM,11/12/2023,114.333333,42.130952,87.321429,48.367857,12.023810,33.452381,35.876190,...,80.221429,11.214286,32.523810,43.738095,23.321429,9.904762,4.297619,16.297619,21.595238,0.261905
285,GSW,MIN,11/12/2023,114.274074,41.000000,90.600000,45.298519,13.748148,38.570370,35.715556,...,77.085926,13.548148,33.303704,46.851852,27.222222,7.562963,3.837037,14.451852,20.918519,2.651852


In [309]:
# now, we need to through the weighted average game stats
# and for each row:
# we need to locate the weighted average game stats of the opposing team, up until that date
# and get the difference, then use the difference as the set of features for the final dataset

# merge the dataset on itself (self-merge)

self_merged = pd.merge(
    left=rolling_weighted_average_data,
    right=rolling_weighted_average_data,
    left_on=["opposing_team", "Game Date"],
    right_on=["Team", "Game Date"],
    suffixes=["", "_opponent"]
)

# get names of columns to take difference
team_stats_cols = game_stats_columns
opponent_stats_cols = [col + "_opponent" for col in game_stats_columns]

# take the difference between the teams
diff_weighted_average = self_merged.apply(lambda row: row[team_stats_cols] - row[opponent_stats_cols].values, axis=1)
diff_weighted_average.columns = [col_name + "_wdiff" for col_name in diff_weighted_average.columns]
diff_weighted_average.tail(20)

,PTS_wdiff,FGM_wdiff,FGA_wdiff,FG%_wdiff,3PM_wdiff,3PA_wdiff,3P%_wdiff,FTM_wdiff,FTA_wdiff,FT%_wdiff,OREB_wdiff,DREB_wdiff,REB_wdiff,AST_wdiff,STL_wdiff,BLK_wdiff,TOV_wdiff,PF_wdiff,+/-_wdiff
266,8.068783,5.406085,7.333333,1.916270,5.444444,11.126984,3.700265,-8.187831,-9.092593,-3.096429,-1.201058,-3.698413,-4.899471,4.928571,-1.936508,-1.343915,-0.488095,0.646825,-6.179894
267,3.575397,3.615079,3.046296,2.479233,-0.747354,-0.404762,-1.751058,-2.907407,-3.030423,-3.009127,1.505291,-0.087302,1.417989,4.058201,0.996032,2.670635,0.423280,-3.544974,1.177249
268,15.712963,4.055556,1.027778,3.804630,6.685185,10.768519,7.097222,0.916667,1.861111,-0.709259,-0.888889,-1.296296,-2.185185,3.638889,-0.555556,-0.092593,-1.101852,-1.537037,13.601852
269,-0.369312,1.571429,-4.611905,4.513386,-2.248148,-8.058466,1.888016,-1.264021,-0.945503,-3.745450,-4.083862,4.529630,0.445767,-2.281746,0.270370,2.234392,2.060053,-1.073280,8.419577
270,10.273810,6.166667,0.190476,6.689286,-4.630952,-9.190476,-3.894048,2.571429,2.928571,0.884524,-0.976190,-1.547619,-2.523810,2.904762,-0.916667,3.738095,1.023810,4.166667,-10.619048
271,0.083333,0.148148,-2.455556,1.362778,-0.642593,-1.348148,-0.544444,0.429630,-0.877778,5.691667,1.000000,4.233333,5.233333,4.077778,-3.733333,0.814815,5.829630,6.040741,-0.872222
272,3.666667,1.798942,-4.197090,4.192989,-0.119048,-1.533069,0.660979,0.187831,1.414021,-3.574074,-4.873016,2.554233,-2.318783,3.489418,-2.046296,1.240741,-2.387566,-1.310847,-2.351852
273,3.222222,3.824074,6.629630,0.466667,-0.620370,0.861111,-3.571296,-3.805556,-2.083333,-9.908333,-0.185185,-1.351852,-1.537037,3.685185,-1.601852,2.462963,-1.472222,2.527778,-12.333333
274,-3.666667,-1.798942,4.197090,-4.192989,0.119048,1.533069,-0.660979,-0.187831,-1.414021,3.574074,4.873016,-2.554233,2.318783,-3.489418,2.046296,-1.240741,2.387566,1.310847,2.351852
275,-15.712963,-4.055556,-1.027778,-3.804630,-6.685185,-10.768519,-7.097222,-0.916667,-1.861111,0.709259,0.888889,1.296296,2.185185,-3.638889,0.555556,0.092593,1.101852,1.537037,-13.601852


## Final Dataset for Training Model (using created features)

Not Yet Done...

In [310]:
# create final dataset with created features
final_dataset = pd.concat([before["home_game"], diff_weighted_average, stability_score_feature, before["W/L"]], axis = 1)

# remove the rows with NA or NaN (rows with no antecedent to compute a rolling average)
final_dataset = final_dataset.dropna()

final_dataset.head()

,home_game,PTS_wdiff,FGM_wdiff,FGA_wdiff,FG%_wdiff,3PM_wdiff,3PA_wdiff,3P%_wdiff,FTM_wdiff,FTA_wdiff,...,DREB_wdiff,REB_wdiff,AST_wdiff,STL_wdiff,BLK_wdiff,TOV_wdiff,PF_wdiff,+/-_wdiff,stability_score,W/L
54,0,14.333333,3.666667,-10.000000,7.866667,5.333333,-1.666667,18.666667,1.666667,2.000000,...,-12.666667,-16.000000,3.333333,1.000000,-4.000000,-3.666667,2.666667,-2.000000,0.946836,L
55,1,4.000000,2.666667,-8.000000,6.433333,1.000000,-9.333333,10.466667,-2.333333,-3.000000,...,6.000000,5.333333,7.333333,-4.333333,4.000000,6.000000,7.000000,13.666667,0.742398,W
57,1,-18.666667,-5.000000,-8.666667,-1.500000,-4.000000,-5.000000,-6.266667,-4.666667,-4.000000,...,9.000000,4.666667,-7.000000,-0.333333,1.000000,8.333333,-1.333333,2.000000,1.514163,W
58,0,18.666667,5.000000,8.666667,1.500000,4.000000,5.000000,6.266667,4.666667,4.000000,...,-9.000000,-4.666667,7.000000,0.333333,-1.000000,-8.333333,1.333333,-2.000000,0.957168,L
60,1,-35.000000,-16.000000,-14.333333,-9.166667,-6.333333,1.333333,-15.533333,3.333333,6.666667,...,-4.666667,-6.333333,-16.333333,0.000000,-0.666667,0.666667,-4.666667,-24.666667,0.976892,L


## Training Predictive Model on Final Dataset

In [311]:
# get data
X = final_dataset.iloc[:, :-1]
y = final_dataset.iloc[:, -1]

In [312]:
from sklearn.model_selection import train_test_split

In [ ]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

,home_game,PTS_wdiff,FGM_wdiff,FGA_wdiff,FG%_wdiff,3PM_wdiff,3PA_wdiff,3P%_wdiff,FTM_wdiff,FTA_wdiff,...,OREB_wdiff,DREB_wdiff,REB_wdiff,AST_wdiff,STL_wdiff,BLK_wdiff,TOV_wdiff,PF_wdiff,+/-_wdiff,stability_score
123,1,10.722222,4.444444,-1.666667,5.922222,0.111111,-4.444444,4.483333,1.722222,-1.055556,...,-2.055556,0.833333,-1.222222,2.666667,0.055556,-1.055556,-0.277778,3.111111,18.277778,0.649605
139,0,-12.600000,0.144444,6.711111,-3.588889,-0.077778,4.411111,-5.648889,-12.811111,-13.666667,...,3.488889,-1.222222,2.266667,0.800000,-1.655556,-5.444444,1.788889,0.555556,-12.522222,0.297690
168,1,-6.833333,-1.555556,-7.722222,2.183333,-0.944444,-8.111111,4.772222,-2.777778,-4.055556,...,-2.333333,-4.277778,-6.611111,1.111111,0.277778,-2.555556,1.888889,1.500000,-7.888889,0.872235
175,0,-2.366667,2.100000,2.800000,0.840000,-9.366667,-16.633333,-8.063333,2.800000,1.500000,...,1.766667,-1.366667,0.400000,1.600000,0.600000,2.900000,1.133333,4.800000,-9.366667,0.901956
105,0,-6.333333,-1.777778,0.777778,-2.877778,1.777778,2.333333,3.044444,-4.555556,-2.555556,...,-1.555556,-4.666667,-6.222222,3.111111,1.555556,-0.555556,4.222222,0.666667,-27.000000,0.927173
